# **Predicting English word version of numbers using RNN**

## **Data**

In [0]:
from fastai.text import *

In [0]:
bs = 64

In [3]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[PosixPath('/root/.fastai/data/human_numbers/valid.txt'),
 PosixPath('/root/.fastai/data/human_numbers/train.txt')]

Train.txt gives us a sequence of numbers written as words.

In [0]:
def readnums(d): 
  return [', '.join(o.strip() for o in open(path/d).readlines())]

In [5]:
train_txt = readnums('train.txt')
train_txt[0][:50]

'one, two, three, four, five, six, seven, eight, ni'

In [6]:
valid_txt = readnums('valid.txt')
valid_txt[0][:40]

'eight thousand one, eight thousand two, '

In [0]:
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)

src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs, bptt=40)

**bptt** (back-propagation through time),  tells us how many steps of history we are considering.

In [8]:
data.bptt

40

In [9]:
len(data.valid_dl)

6

In [0]:
v = data.valid_ds.vocab

In [11]:
nv = len(v.itos)
nv

40

In [12]:
v.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 ',',
 'hundred',
 'thousand',
 'one',
 'two',
 'three',
 'four',
 'five',
 'six',
 'seven',
 'eight',
 'nine',
 'twenty',
 'thirty',
 'forty',
 'fifty',
 'sixty',
 'seventy',
 'eighty',
 'ninety',
 'ten',
 'eleven',
 'twelve',
 'thirteen',
 'fourteen',
 'fifteen',
 'sixteen',
 'seventeen',
 'eighteen',
 'nineteen',
 'xxfake']

In [13]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 40]), torch.Size([64, 40]))

In [14]:
data.show_batch()

idx,text
0,"four , thirty five , thirty six , thirty seven , thirty eight , thirty nine , forty , forty one , forty two , forty three , forty four , forty five , forty six , forty seven ,"
1,"twenty eight , two hundred twenty nine , two hundred thirty , two hundred thirty one , two hundred thirty two , two hundred thirty three , two hundred thirty four , two hundred thirty five , two hundred thirty"
2,"three hundred ninety two , three hundred ninety three , three hundred ninety four , three hundred ninety five , three hundred ninety six , three hundred ninety seven , three hundred ninety eight , three hundred ninety nine ,"
3,"fifty nine , five hundred sixty , five hundred sixty one , five hundred sixty two , five hundred sixty three , five hundred sixty four , five hundred sixty five , five hundred sixty six , five hundred sixty"
4,", seven hundred twenty seven , seven hundred twenty eight , seven hundred twenty nine , seven hundred thirty , seven hundred thirty one , seven hundred thirty two , seven hundred thirty three , seven hundred thirty four ,"


In [0]:
nh=56

## **Adding a GRU**

In [0]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh, nh, 1, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [17]:
nv, nh

(40, 56)

In [0]:
learn = Learner(data, Model5(), metrics=accuracy)

In [19]:
learn.fit_one_cycle(15, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.603911,3.505888,0.179102,00:00
1,3.052274,2.267071,0.314779,00:00
2,2.368582,2.095855,0.323763,00:00
3,1.934072,2.016207,0.372070,00:00
4,1.599120,1.970902,0.536914,00:00
5,1.304959,1.798961,0.570247,00:00
6,1.038446,1.800910,0.584896,00:00
7,0.796628,1.798545,0.623958,00:00
8,0.601719,1.861906,0.632357,00:00
9,0.453580,1.821679,0.627148,00:00
